<a href="https://colab.research.google.com/github/computational-neurology/workshop2025/blob/main/04_fitting_prep_material.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fitting

### Learning goals
- explain why model fitting is needed
- perform a fitting procedure for a two-node problem
- explain appropriate fitting methods for brain networks

Imagine you have a dataset and you want to simulate the time series. How do you make sure that the parameters are chosen in such a way that they represent the data best? Let's try out the manual way first.

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install git+https://github.com/neurolib-dev/neurolib.git
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install nilearn


# @title Run this to install dependencies

In [ ]:
# Let's import all the relevant libraries
from neurolib.models.hopf import HopfModel
import numpy as np
import matplotlib.pyplot as plt
from nilearn.connectome import ConnectivityMeasure

<b>Exercise </b><p>

### Fitting different time series

You will be given two sets of time series with 2 regions that we created used two different sets of model parameters. You will be given a list of all the parameters used except for one. You have to recreate these time series by adjusting the model parameters G (the coupling parameter) in Model A and Omega (the instrinsic frequency) in Model B. Your goal is to choose the model parameter of both Model A and B in such a way that the time series that you create look as close as possible to the ones you are provided with. The process to find the parameters that were used to generate artificial data is called parameter recovery.

You will find the time series for Model A as model_a.npy, and so on. Please first find your own solution and write it down and then compare it with the others and write down the adapted solution after the discussion. In A and B, since there are only two nodes, you can guess the best parameter based on how similar your simulated timeseries are to the empirical ones. In C you should try to fit the resulting functional connectivity matrix.

You will be given a range of possible parameters. Try out the best fitting strategy and reflect on how you proceeded. To ensure that the starting point is the same as with the artificial data, please params["seed"] = 42 in your model.

## Model parameters Model A:
### Known parameters:
   
      w:  0.7 * np.pi
      Sigma_ou: 0.1
      Cmat: np.array([[1, 0.02], [0.02, 1]])
      Dmat: 0 for each connection
      Duration: 50ms

### Parameter search for:
            
      G (between 0.5 and 2.5)

## Model parameters Model B:
### Known parameters:

      G: 0.5
      Sigma_ou: 0.1
      Cmat: np.array([[1, 0.02], [0.02, 1]])
      Dmat: 0 for each connection
      Duration: 50ms

### Parameter search for:
      
      Omega (between 0.1 and 1.8)


#### Write down your answer here:
#### Model A:
Group =

Own solution for G =

Adapted solution for G after discussion =

#### Model B:
Group =

Own solution for Omega =

Adapted solution for Omega after discussion =


### What was your strategy on finding the right parameter?

ANSWER:


Rate the efficiency and complexity of the strategy:

ANSWER:


If you were to program a strategy, how would it look like?

ANSWER:


How would you adapt the strategy if you had more nodes?

ANSWER:
<p>
 -End of exercise-
    </div>

In [ ]:
# solution to exercise here:

## Further reading material
### Graph theory
Olaf Sporns. Networks of the Brain. MIT Press, 2010
Marcus Kaiser. A tutorial on connectome analysis, 2011. https://www.dynamic-connectome.org/pubs/Kaiser2011Neuroimage.pdf

### Model fitting
A tutorial on model fitting. https://training.incf.org/lesson/model-fitting-i-intro-lecture